In [1]:
import os
import sqlite3
import pandas as pd

In [2]:
os.chdir('...')

In [3]:
# Assuming errors will be encountered, this will allow to code to continue running without errors
try:
    os.remove('airline2.db')
except OSError:
        pass

In [4]:
conn = sqlite3.connect('airline2.db')

In [5]:
airports = pd.read_csv("./Block 3/airports.csv")
carriers = pd.read_csv("./Block 3/carriers.csv")
planes = pd.read_csv("./Block 3/plane-data.csv")

airports.to_sql('airports', con = conn, index = False)
carriers.to_sql('carriers', con = conn, index = False)
planes.to_sql('planes', con = conn, index = False)

In [6]:
c = conn.cursor()

In [7]:
# The creation of the table helps with the warning message about the mixed data types
c.execute('''
CREATE TABLE ontime (
 Year int,
 Month int,
 DayofMonth int,
 DayofWeek int,
 DepTime int,
 CRSDepTime int,
 ArrTime int,
 CRSArrTime int,
 UniqueCarrier varchar(5),
 FlightNum int,
 TailNum varchar(8),
 ActualElapsedTime int,
 CRSElapsedTime int,
 AirTime int,
 ArrDelay int,
 DepDelay int,
 Origin varchar(3),
 Dest varchar(3),
 Distance int,
 TaxiIn int,
 TaxiOut int,
 Cancelled int,
 CancellationCode varchar(1),
 Diverted varchar(1),
 CarrierDelay int,
 WeatherDelay int,
 NASDelay int,
 SecurityDelay int,
 LateAircraftDelay int
)
''')
conn.commit()

In [8]:
# sqlite_master is a built in SQLite metadata table
c.execute('''
SELECT name
FROM sqlite_master
WHERE type = 'table'
''').fetchall()

[('airports',), ('carriers',), ('planes',), ('ontime',)]

In [9]:
for year in range(2000, 2006):
    filename = "./Block 3/"+str(year)+".csv"
    print('Processing:',year)
    ontime = pd.read_csv(filename, dtype = {'Cancelled':int}) # In order to change the data type to integers
    ontime.to_sql('ontime', con = conn, if_exists = 'append', index = False)

conn.commit()

Processing: 2000
Processing: 2001
Processing: 2002
Processing: 2003


C:\Users\Victor\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Processing: 2004
Processing: 2005


In [12]:
# Query 1
c.execute('''
SELECT model AS model, AVG(ontime.DepDelay) AS avg_delay
FROM planes JOIN ontime ON planes.tailnum=ontime.TailNum
WHERE ontime.Cancelled = 0 AND ontime.Diverted = 0 AND ontime.DepDelay > 0
GROUP BY model
ORDER BY avg_delay
''')

x = c.fetchall()
print(x[0][0], "has the lowest associated average departure delay at", round(x[0][1],3))

737-2Y5 has the lowest associated average departure delay at 7.022


In [18]:
x[0][0]

'737-2Y5'

In [16]:
round(x[0][1],3)

7.022

In [20]:
# To show the list
x[:]

[('737-2Y5', 7.022026431718062),
 ('737-282', 8.433566433566433),
 ('737-230', 10.458646616541353),
 ('737-282C', 11.765765765765765),
 ('767-324', 15.788051209103841),
 ('767-432ER', 15.793831536083252),
 ('757-26D', 15.942548158161541),
 ('182A', 17.10326086956522),
 ('A318-111', 17.147109826589595),
 ('767-3P6', 17.385817307692307),
 ('757-212', 17.51070110701107),
 ('777-232', 17.521508794236066),
 ('737-832', 17.69418506410021),
 ('757-232', 17.774470060792478),
 ('767-332', 17.81414698606352),
 ('A319-111', 18.67868088758122),
 ('DA 20-A1', 20.041798606713108),
 ('767-3CB', 20.716778523489932),
 ('EMB-120', 20.798148274218804),
 ('757-351', 21.41301447699985),
 ('EMB-120ER', 21.45844681026657),
 ('A319-132', 21.631674801486124),
 ('767-224', 21.835771105978957),
 ('A321-211', 21.986651835372637),
 ('MD-90-30', 22.20332326283988),
 ('A320-214', 22.524075366364272),
 ('747-451', 22.589041095890412),
 ('737-3S3', 22.61637406570485),
 ('767-33A', 22.61645218268988),
 ('777-224', 22.7

In [21]:
# Query 2
c.execute('''
SELECT airports.city AS city, COUNT(*) AS total
FROM airports JOIN ontime ON ontime.dest = airports.iata
WHERE ontime.Cancelled = 0
GROUP BY airports.city
ORDER BY total DESC
''')

x = c.fetchall()
print(x[0][0],"has the highest number of inbound flights(excluding canceled flights) amounting to", x[0][1])

Chicago has the highest number of inbound flights(excluding canceled flights) at 2388365


In [22]:
x[:5]

[('Chicago', 2388365),
 ('Atlanta', 1912370),
 ('Dallas-Fort Worth', 1781833),
 ('Houston', 1304407),
 ('Los Angeles', 1279514)]

In [23]:
# Query 3
c.execute('''
SELECT carriers.Description AS carrier, COUNT(*) AS total
FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.code
WHERE ontime.Cancelled = 1
GROUP BY carriers.Description
ORDER BY total DESC
''')

x = c.fetchall()
print(x[0][0], "has the highest number of cancelled flights amounting to", x[0][1])

Delta Air Lines Inc. has the highest number of cancelled flights 107851


In [24]:
x[:5]

[('Delta Air Lines Inc.', 107851),
 ('American Airlines Inc.', 105762),
 ('United Air Lines Inc.', 102066),
 ('American Eagle Airlines Inc.', 96365),
 ('US Airways Inc. (Merged with America West 9/05. Reporting for both starting 10/07.)',
  85866)]

In [27]:
# Query 4(Long version)
c.execute('''
SELECT
q1.carrier AS carrier, (CAST(q1.numerator AS FLOAT)/ CAST(q2.denominator AS FLOAT)) AS ratio
FROM
 (SELECT carriers.Description AS carrier, COUNT(*) AS numerator
  FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.Code
  WHERE ontime.Cancelled = 1
  GROUP BY carriers.Description
  )
  AS q1 JOIN
  (SELECT carriers.Description AS carrier, COUNT(*) AS denominator
  FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.code
  GROUP BY carriers.Description
  ) 
  AS q2 USING (carrier)
  ORDER BY ratio DESC
''')

x = c.fetchall()
print(x[0][0], "has the highest number of canceled flights, relative to their total number of flights", round(x[0][1],3))

American Eagle Airlines Inc. has the highest number of canceled flights, relative to their total number of flights 0.041


In [28]:
x[:5]

[('American Eagle Airlines Inc.', 0.04059832660661774),
 ('Comair Inc.', 0.03310794068502934),
 ('Atlantic Southeast Airlines', 0.032601899162408715),
 ('Independence Air', 0.03199782987301006),
 ('United Air Lines Inc.', 0.027924193799912997)]

In [30]:
# Query 4(Short version)
c.execute('''
SELECT carriers.Description, AVG(Cancelled) AS Ratio
FROM carriers INNER JOIN ontime ON ontime.UniqueCarrier = carriers.Code
GROUP BY carriers.Description
ORDER BY Ratio DESC
''')

x= c.fetchall()
print(x[0][0], "has the highest number of canceled flights, relative to their total flights of", round(x[0][1],3))

American Eagle Airlines Inc. has the highest number of canceled flights, relative to their total flights 0.041


In [31]:
x[:5]

[('American Eagle Airlines Inc.', 0.04059832660661774),
 ('Comair Inc.', 0.03310794068502934),
 ('Atlantic Southeast Airlines', 0.032601899162408715),
 ('Independence Air', 0.03199782987301006),
 ('United Air Lines Inc.', 0.027924193799912997)]

In [32]:
conn.close()